# Offloading workload from Cloud Pak for Data to Watson ML Accelerator

Using the Watson Machine Learning API Python client library. You will learn to download a sample Pytorch MNIST model and train it using Watson Machine Learning Accelerator (WMLA) to identify numbers based on images. Then, you will save and deploy the model from your workspace and again use WML to score it. Lastly you will clean up by deleting your deployment. This notebook runs on Python 3.6 and is intended for CPD 3.0.1.

## Table of Contents:

1. [Setup](#setup)<br>
    1.1 [Initialize python client](#Initialize-python-client)<br>
    1.2 [Set default space](#Set-default-space)<br>
    1.3 [Create library](#Create-library)<br>
2. [Train the model](#train)<br>
    2.1 [Create training](#Create-training)<br>
    2.2 [Monitor training](#Monitor-training)<br>
3. [Save and deploy the model](#deploy)<br>
    3.1 [Save Model to project](#Save-Model-to-project)<br>
    3.2 [Create Deployment](#Create-Deployment)<br>
4. [Scoring the model](#score)<br>
    4.1 [Score deployment](#Score-deployment)<br>
    4.2 [Prediction accuracy](#Prediction-accuracy)<br>
    4.3 [Interact with wider project](#Interact-with-wider-project)<br>
5. [Clean up resources](#Clean-up-resources)

&nbsp;
&nbsp;
&nbsp;
&nbsp;
&nbsp;


![options](https://github.com/IBM/wmla-learning-path/raw/master/shared-images/CPD-WMLA_3_pythonclient.png)

<a id = "setup"></a>
## Setup

Uncomment and run this cell if Python v4 client is not installed.

In [ ]:
!pip install --upgrade watson-machine-learning-client-V4

### Initialize python client

In [1]:
import sys,os,os.path

Authentication methods:

- If you use the same user ID on both CPD and WMLA, use the **token** authentication method
- If you use different user IDs on CPD and WMLA, use the **user** authentication

Update username and password to access WMLA in cell below, plus the URL for local CPD cluster with WML:

In [2]:
# @hidden_cell

# Authenticate using token for a user which exists on both the CPD and WMLA cluster.
# Otherwise specify username and password

# token = os.environ['USER_ACCESS_TOKEN']
USERNAME = 'wml-user'
PASSWORD = 'password'
URL = '' # Enter URL


wml_credentials = {
#     "token": token,
    'username': USERNAME,
    'password': PASSWORD,
    "instance_id" : "wml_local",
    "url": URL,
    "version": "3.0.1"
}

#credential designs
#version would be corresponding to CPD or Openshift version

In [3]:
# This can take time to run - be patient :)
# Creating an instance of python client

from watson_machine_learning_client import WatsonMachineLearningAPIClient
client = WatsonMachineLearningAPIClient(wml_credentials)
client.version

'1.0.108'

### Set default space

Set default space for CPD usage (if needed, note this unsets the project ID). 

In [4]:
# creating deployment space
meta_props = {
    client.spaces.ConfigurationMetaNames.NAME: "CPD WMLA demo space"
}

space = client.spaces.store(meta_props)
space_id = client.spaces.get_uid(space)

print("Space id: {}".format(space_id))
# all assets creating after this cell will be stored at this space_id
client.set.default_space(space_id)

Space id: c3307924-a53a-47fd-b11c-10b2f0455c60


'SUCCESS'

In [5]:
client.spaces.list()

------------------------------------  -------------------  ------------------------
GUID                                  NAME                 CREATED
c3307924-a53a-47fd-b11c-10b2f0455c60  CPD WMLA demo space  2020-07-24T20:11:49.866Z
0a1617b5-52eb-4de2-9758-3c2d62985aa9  CPD WMLA demo space  2020-07-24T13:37:12.803Z
5eb19bcb-5ac5-4597-9665-438ea4f09a12  CPD WMLA demo space  2020-07-23T14:19:38.376Z
d516a0bc-4d92-4d8f-9ac3-de8e869abd09  CPD WMLA demo space  2020-07-23T14:19:26.218Z
d91c86e3-000c-4156-a738-4b5baf630b83  wml_training_space   2020-07-16T13:24:30.254Z
e9fde3b1-c2a6-4066-9c7f-98f379532455  CPD WMLA demo space  2020-07-14T17:11:58.060Z
5527ef2f-6367-48aa-b672-b8a3c3213a1e  CPD WMLA demo space  2020-07-13T20:47:32.891Z
92d54f50-7858-4089-9574-6841abdc29db  CPD WMLA demo space  2020-07-13T20:45:21.322Z
a4f928ca-49b4-493e-97f4-09f85ca70090  CPD WMLA demo space  2020-07-13T18:29:02.548Z
38340881-7541-41ab-ba78-82b11cb6f8fe  CPD WMLA demo space  2020-07-13T14:59:00.297Z
---------

In [ ]:
# In case you need to clear up any old spaces before this session

# client.spaces.delete('<GUID_HERE>')

View the predefined runtimes available.

In [6]:
# retrieve existing runtime in CPD cluster,  where training and deployment will happen
client.runtimes.list(pre_defined=True, limit=100)

WARNING!! 'runtimes' is DEPRECATED. Use 'software_specifications' instead to create and manage runtimes/specifications
WARNING!! 'runtimes' is DEPRECATED. Use 'software_specifications' instead to create and manage runtimes/specifications
--------------------------  --------------------------  ------------------------  --------
GUID                        NAME                        CREATED                   PLATFORM
do_12.10                    do_12.10                    2020-07-07T05:30:57.666Z  do
do_12.9                     do_12.9                     2020-07-07T05:30:57.632Z  do
pmml_4.3                    pmml_4.3                    2020-07-07T05:30:57.609Z  pmml
pmml_4.2.1                  pmml_4.2.1                  2020-07-07T05:30:57.585Z  pmml
pmml_4.2                    pmml_4.2                    2020-07-07T05:30:57.559Z  pmml
pmml_4.1                    pmml_4.1                    2020-07-07T05:30:57.534Z  pmml
pmml_4.0                    pmml_4.0                    2020-0

### Create library

Get the zip file containing the training python code is downloaded. It can be modified to use locally stored files.

We are using a pre-created sample zip file which contains the following: 
```
pytorch_onnx_v_1.1
├── emetrics.py
└── pytorch_v_1.1_mnist_onnx.py
```

This code trains a PyTorch model using the MNIST dataset which is downloaded to the WMLA cluster as part of the workload running. 

In [7]:
import requests

model_content_resp = requests.get("https://github.com/calinrc/model_definitions/raw/master/libs/pytorch_onnx_v_1.1.zip",
                                  headers={"Content-Type": "application/octet-stream"})
with open("pytorch_onnx_v_1.1.zip", "wb") as f:
    f.write(model_content_resp.content)

In [ ]:
#client.runtimes.get_library_details(limit=10)

In [8]:
# stores model definition as custom library, and we will use for training.
meta_props = {
    client.model_definitions.ConfigurationMetaNames.NAME: "pytorch mnist library 4",
    #client.model_definitions.ConfigurationMetaNames.FILEPATH: "./pytorch_onnx_v_1.1.zip",
    client.model_definitions.ConfigurationMetaNames.PLATFORM: {"name": "python",
                                               "versions": ["3.6"]},
    client.model_definitions.ConfigurationMetaNames.VERSION: "1",
    client.model_definitions.ConfigurationMetaNames.COMMAND: "pytorch_v_1.1_mnist_onnx.py --debug-level debug"
}

model_definition_details = client.model_definitions.store("./pytorch_onnx_v_1.1.zip",meta_props)

model_definition_id = client.model_definitions.get_uid(model_definition_details)

In [ ]:
#library_href = client.runtimes.get_library_href(library)

<a id = "train"></a>
## Train the model

### Create training

In [9]:
# Create Training pipeline and use the model definition stored in the library
# Command doesn't require to be defined here.   
# Library is deprecated from CPD 3.0.1
meta_props = {
    client.training.ConfigurationMetaNames.NAME: model_definition_id,
    client.training.ConfigurationMetaNames.DESCRIPTION: "PyTorch training at WMLA",
    client.training.ConfigurationMetaNames.MODEL_DEFINITION: {
     "id": model_definition_id,
     #"href": library_href,
     "runtime": {"href": "/v4/runtimes/pytorch-onnx_1.1-py3.6"},
     #"command": "pytorch_v_1.1_mnist_onnx.py --epochs 10 --debug-level debug",   -> will override training command stored in library
     "software_spec": {
      "name": "pytorch-onnx_1.1-py3.6"
     },   
     
     "hardware_spec": {"name": "v100", "nodes": 1},
     "parameters": {
            "name": "pytorch onnx defintion",
            "description": "pytorch onnx defintion"
          }
     },
    client.training.ConfigurationMetaNames.SPACE_UID: space_id,    
    
    client.training.ConfigurationMetaNames.TRAINING_DATA_REFERENCES: [
        {
          "name": "training_input_data",
          "type": "fs",
          "connection": {},
          "location": {
            "path": "pytorch-mnist"
          },
        }
      ],
    client.training.ConfigurationMetaNames.TRAINING_RESULTS_REFERENCE: {
        "location": {
          "path": "/spaces/" + space_id + "/assets/trainings"
        },
        "type": "fs"
      }
}

training = client.training.run(meta_props)


In [10]:
training_uid = client.training.get_uid(training)
training_state = client.training.get_status(training_uid)['state']
print("Training uid: {}".format(training_uid))

Training uid: 38beda87-6b1c-4dcf-bd53-4dbc3da02182


In [11]:
client.training.get_details(training_uid)

{'metadata': {'created_at': '2020-07-24T20:12:22.131Z',
  'description': 'PyTorch training at WMLA',
  'guid': '38beda87-6b1c-4dcf-bd53-4dbc3da02182',
  'href': '/v4/trainings/38beda87-6b1c-4dcf-bd53-4dbc3da02182',
  'id': '38beda87-6b1c-4dcf-bd53-4dbc3da02182',
  'modified_at': '2020-07-24T20:12:33.747Z',
  'name': '2059862f-fb24-4416-b415-0686e587fc5d',
  'space_id': 'c3307924-a53a-47fd-b11c-10b2f0455c60'},
 'entity': {'description': 'PyTorch training at WMLA',
  'model_definition': {'hardware_spec': {'name': 'v100'},
   'id': '2059862f-fb24-4416-b415-0686e587fc5d',
   'parameters': {'description': 'pytorch onnx defintion',
    'name': 'pytorch onnx defintion'},
   'software_spec': {'name': 'pytorch-onnx_1.1-py3.6'}},
  'name': '2059862f-fb24-4416-b415-0686e587fc5d',
  'results_reference': {'location': {'path': '/spaces/c3307924-a53a-47fd-b11c-10b2f0455c60/assets/trainings',
    'model': '/spaces/c3307924-a53a-47fd-b11c-10b2f0455c60/assets/trainings/38beda87-6b1c-4dcf-bd53-4dbc3da021

### Monitor training

In [12]:
import time 

while training_state == 'pending' or training_state == 'running':
    time.sleep(10)
    print("Current training state: {}".format(training_state))
    training_state = client.training.get_status(training_uid)['state']

print ("training completes")    

Current training state: pending
Current training state: running
Current training state: running
Current training state: running
Current training state: running
Current training state: running
Current training state: running
Current training state: running
Current training state: running
Current training state: running
Current training state: running
Current training state: running
Current training state: running
Current training state: running
Current training state: running
Current training state: running
training completes


<a id = "deploy"></a>
## Save and deploy the model

### Save Model to project

Note that you can save your model to your Watson Studio project by changing from the space we've been working in up to this point by using the `PROJECT_ID` variable. 

This saved model can then be promoted to the CPD deployment space from the UI (not currently possible with python client).

In [ ]:
# Only use this if you wish to save model to Watson Studio project. Promotion to deployment space / deployment must then be done from UI.

project_id = os.environ['PROJECT_ID']
client.set.default_project(project_id)

Otherwise stick with the working space you've been using so far:

In [ ]:
# Only use this if you need to change back to default space created in this notebook

client.set.default_space(space_id)

In [ ]:
meta_props = {
    client.repository.ModelMetaNames.NAME: "Pytorch MNIST model demo",
    client.repository.ModelMetaNames.TRAINING_DATA_REFERENCES: [client.training.get_details(training_uid)["entity"]["results_reference"]],
    client.repository.ModelMetaNames.RUNTIME_UID: "pytorch-onnx_1.1-py3.6",
    client.repository.ModelMetaNames.TYPE: "pytorch-onnx_1.1"
}

model = client.repository.store_model(training_uid, meta_props)

In [ ]:
model_uid = client.repository.get_model_uid(model)

In [ ]:
client.repository.list_models()

### Create Deployment

Read more on deploying models here: https://www.ibm.com/support/producthub/icpdata/docs/content/SSQNUZ_current/wsj/wmls/wmls-deploy-python.html

In [ ]:
client.deployments.get_details()

In [ ]:
meta_props = {
    client.deployments.ConfigurationMetaNames.NAME: "Pytorch MNIST deployment",
    client.deployments.ConfigurationMetaNames.ONLINE: {}
}

deployment = client.deployments.create(model_uid, meta_props)

In [ ]:
deployment_uid = client.deployments.get_uid(deployment)
print("Deployment uid: {}".format(deployment_uid))

<a id = "score"></a>
## Scoring the model

### Score deployment

In [ ]:
values = [[[[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0.6392157077789307,0.7568628191947937,0.5960784554481506,0.3607843220233917,0.20000001788139343,0.20000001788139343,0.20000001788139343,0.20000001788139343,0.12156863510608673,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0.08235294371843338,0.874509871006012,0.9921569228172302,0.988235354423523,0.9921569228172302,0.988235354423523,0.9921569228172302,0.988235354423523,0.9921569228172302,0.7529412508010864,0.32156863808631897,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0.20000001788139343,0.9921569228172302,0.40000003576278687,0,0.08235294371843338,0.40000003576278687,0.24313727021217346,0.40000003576278687,0.40000003576278687,0.2392157018184662,0.7176470756530762,0.1568627506494522,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0.20000001788139343,0.988235354423523,0.40000003576278687,0,0,0,0,0,0,0,0.2392157018184662,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0.20000001788139343,0.9921569228172302,0.40000003576278687,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0.3607843220233917,0.988235354423523,0.40000003576278687,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0.6784313917160034,0.9921569228172302,0.40000003576278687,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0.9921569228172302,0.988235354423523,0.874509871006012,0.7960785031318665,0.7960785031318665,0.7960785031318665,0.32156863808631897,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0.48235297203063965,0.9960784912109375,0.9921569228172302,0.8784314393997192,0.7960785031318665,0.7960785031318665,0.874509871006012,0.9960784912109375,0.27843138575553894,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0.16078431904315948,0.9529412388801575,0.9921569228172302,0.5098039507865906,0.0784313753247261,0,0,0.0784313753247261,0.9921569228172302,0.9098039865493774,0.16078431904315948,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0.5960784554481506,0.9921569228172302,0.7176470756530762,0,0,0,0,0,0.5176470875740051,0.9921569228172302,0.40000003576278687,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0.20000001788139343,0.5921568870544434,0.0784313753247261,0,0,0,0,0,0.20000001788139343,0.988235354423523,0.40000003576278687,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0.08235294371843338,0,0,0,0,0,0,0,0.4431372880935669,0.9921569228172302,0.40000003576278687,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0.32156863808631897,0.7176470756530762,0,0,0,0,0,0,0,0.7568628191947937,0.988235354423523,0.40000003576278687,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0.7960785031318665,0.7176470756530762,0,0,0,0,0,0,0.08235294371843338,0.9960784912109375,0.9921569228172302,0.16078431904315948,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0.08235294371843338,0.874509871006012,0.40000003576278687,0,0,0,0,0,0.08235294371843338,0.7960785031318665,0.9921569228172302,0.5098039507865906,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0.8000000715255737,0.48235297203063965,0,0,0,0,0.16078431904315948,0.6784313917160034,0.9921569228172302,0.7960785031318665,0.0784313753247261,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0.6352941393852234,0.874509871006012,0.40000003576278687,0.08235294371843338,0.40000003576278687,0.6392157077789307,0.9529412388801575,0.9921569228172302,0.6705882549285889,0.0784313753247261,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0.16078431904315948,0.917647123336792,0.9921569228172302,1,0.9921569228172302,1,0.6745098233222961,0.32156863808631897,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0.11764706671237946,0.5137255191802979,0.7529412508010864,0.43529415130615234,0.19607844948768616,0.03921568766236305,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]]]]

meta_props = {
    client.deployments.ScoringMetaNames.INPUT_DATA: [{"values": values}]
}    
predictions = client.deployments.score(deployment_uid, meta_props)
print("Predictions returned\n{}".format(predictions))

### Prediction accuracy

In [ ]:
highest_log_probability = max(predictions['predictions'][0]['values'][0])
prediction = predictions['predictions'][0]['values'][0].index(highest_log_probability)
print("We predict the picture below is a {}".format(prediction))

from matplotlib import pyplot as plt 
import numpy as np

first_image = np.array(values[0][0], dtype='float')
plt.imshow(first_image, cmap='gray')
plt.show()

### Interact with wider project 

In [ ]:
from project_lib import Project
project = Project.access()
storage_credentials = project.get_storage_metadata()

In [ ]:
project.get_name()

### Clean up resources

In [ ]:
print("Deployment deletion: {}".format(client.deployments.delete(deployment_uid)))
print("Space deletion: {}".format(client.spaces.delete(space_id)))


In [ ]:
print("Space deletion: {}".format(client.spaces.delete('4b8f7e78-81e6-40b3-a6d6-0057f0771695')))

<a id = "summary"></a>
## Summary

Congratulations! You have learned to:

1. Download the Pytorch MNIST model
2. Create a Watson Machine Learning model by using the Pytorch model
3. Train the model by offloading work to Watson Machine Learning Acclerator
4. Save and deploy from your workspace
5. Score the model
6. Clean up